In [1]:
import json
from pathlib import Path
from xsdata.formats.dataclass.parsers import XmlParser, JsonParser, DictDecoder
from xsdata.formats.dataclass.parsers.config import ParserConfig
from xsdata.formats.dataclass.context import XmlContext
import numpy as np
import pandas as pd
#import openaml_importer.model as models

In [2]:
with open("data/assets.json", "r") as f:
    data = json.load(f) 

In [3]:
type(data)

dict

In [4]:
dd = data['assets']

In [5]:
# list of the asset type
l=[]
for i in range(len(dd)):
    l.append(dd[i]['assetType'])
lis = set(l)
print(lis)

{'ETF_BONDS', 'FUTURE_ON_INDEX', 'MONEY_MARKET_INDEX', 'CURRENCY', 'OPTION_ON_INDEX', 'MONEY_MARKET_INDEX_OIS', 'CONVERTIBLE_BOND', 'EQUITY_INDEX', 'BON_MOYEN_TERME', 'FUTURE_ON_MONEY_MARKET', 'EURO_COMMERCIAL_PAPER', 'NEU_CP', 'FX_SWAP', 'FIXED_RATE_BOND', 'CASH_PROVISION', 'INDEX_BOND', 'VARIABLE_BOND', 'REPO', 'MISCELLANOUS_INDEXES', 'MUTUAL_FUND_M_M', 'CRD_INDEX', 'SWAP_ON_RATE', 'ITRAXX', 'FRENCH_TREASURY_NOTE', 'FX_FORWARD', 'FUTURE_ON_BOND', 'COMM_PAPER_REGULAR', 'INFLATION_SWAP', 'US_COMMERCIAL_PAPER', 'CASH_DEPOSIT', 'FUTURE_ON_CURRENCY', 'ETF_EQUITIES', 'CERTIFICAT_DE_DEPOT_NEGOCIABLE', 'TERM_DEPOSIT', 'EQUITY_ORDINARY'}


In [6]:
pdlis = pd.DataFrame(columns=['occurrences'])
for k in lis:
    j=0
    for i in range(len(dd)):
        if (dd[i]['assetType']==k):
            j=j+1
    pdnew=pd.DataFrame([j], columns=['occurrences'], index=[k])
    pdlis = pd.concat([pdlis, pdnew], ignore_index=False).sort_values("occurrences", ascending=False)
pdlis    

,occurrences
FIXED_RATE_BOND,1148
SWAP_ON_RATE,274
NEU_CP,253
COMM_PAPER_REGULAR,195
REPO,117
VARIABLE_BOND,78
BON_MOYEN_TERME,18
FX_FORWARD,17
CASH_DEPOSIT,15
TERM_DEPOSIT,14


In [7]:
# list of swaps ESTR
#pd.set_option('display.max_rows', 300)
col=['maturityDate', 'quotationdate', 'quotationvalue', 'side0', 'ratetype', 'frequency','index','side1','ratetype1','frequency', 'rate'] #
pdswap = pd.DataFrame(columns=col)
j=0
for i in range(len(dd)):
    if ((dd[i]['assetType']=='SWAP_ON_RATE') and ("_ESTR" in dd[i]['name'])):
        #print('tt',dd[i]['legs'][0]['interest'].get('index'))
        pdnew=pd.DataFrame([[dd[i]['maturityDate'],
                             dd[i]['quotations'][0]['date'],
                             dd[i]['quotations'][0]['close'],
                             dd[i]['legs'][0]['side'], 
                             dd[i]['legs'][0]['interest']['rateType'], 
                             dd[i]['legs'][0]['interest']['paymentFrequency']['code'], 
                             #dd[i]['legs'][0]['interest']['dayCountMethod'], 
                             dd[i]['legs'][0]['interest']['index']['_id'], 
                             dd[i]['legs'][1]['side'], 
                             dd[i]['legs'][1]['interest']['rateType'], 
                             dd[i]['legs'][1]['interest']['paymentFrequency']['code'], 
                             #dd[i]['legs'][1]['interest']['dayCountMethod'],
                             dd[i]['legs'][1]['interest'].get('rate')
                            ]], 
                             columns=col, index=[dd[i]['name']])
        pdswap = pd.concat([pdswap, pdnew], ignore_index=False)
display(pdswap)

/tmp/ipykernel_36055/301726119.py:24: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  pdswap = pd.concat([pdswap, pdnew], ignore_index=False)


,maturityDate,quotationdate,quotationvalue,side0,ratetype,frequency,index,side1,ratetype1,frequency,rate
SW20250131_ESTR O/3.246,2025-01-31,2024-12-19,100.378497,RECEIVE,FLOATING,INFI,6741831,PAY,FIXED,INFI,3.2460
SW20250113_ESTR O/2.9599,2025-01-13,2024-12-19,100.000814,RECEIVE,FLOATING,INFI,6741831,PAY,FIXED,INFI,2.9599
SW20250908_ESTR O/2.813,2025-09-08,2024-12-19,99.786749,RECEIVE,FLOATING,INFI,6741831,PAY,FIXED,INFI,2.8130
SW20250131_ESTR O/3.246,2025-01-31,2024-12-19,100.378497,RECEIVE,FLOATING,INFI,6741831,PAY,FIXED,INFI,3.2460
SW20250205_ESTR O/3.0175,2025-02-05,2024-12-19,100.001246,RECEIVE,FLOATING,INFI,6741831,PAY,FIXED,INFI,3.0175
...,...,...,...,...,...,...,...,...,...,...,...
SW20251112_ESTR O/2.3055,2025-11-12,2024-12-19,100.006444,RECEIVE,FLOATING,INFI,6741831,PAY,FIXED,INFI,2.3055
SW20250207_ESTR O/3.0094,2025-02-07,2024-12-19,100.002011,RECEIVE,FLOATING,INFI,6741831,PAY,FIXED,INFI,3.0094
SW20250718_ESTR O/3.23,2025-07-18,2024-12-19,99.631221,RECEIVE,FLOATING,INFI,6741831,PAY,FIXED,INFI,3.2300
SW20250219_ESTR O/2.9164,2025-02-19,2024-12-19,100.011090,RECEIVE,FLOATING,INFI,6741831,PAY,FIXED,INFI,2.9164


In [8]:
# list of swaps SOFR
j=0
pdswap = pd.DataFrame(columns=col)
for i in range(len(dd)):
    if ((dd[i]['assetType']=='SWAP_ON_RATE') and ("SOFR" in dd[i]['name'])):
        j=j+1
        pdnew=pd.DataFrame([[dd[i]['maturityDate'],
                             dd[i]['quotations'][0]['date'],
                             dd[i]['quotations'][0]['close'],
                             dd[i]['legs'][0]['side'], 
                             dd[i]['legs'][0]['interest']['rateType'], 
                             dd[i]['legs'][0]['interest']['paymentFrequency']['code'], 
                             #dd[i]['legs'][0]['interest']['dayCountMethod'], 
                             dd[i]['legs'][0]['interest']['index']['_id'], 
                             dd[i]['legs'][1]['side'], 
                             dd[i]['legs'][1]['interest']['rateType'], 
                             dd[i]['legs'][1]['interest']['paymentFrequency']['code'], 
                             #dd[i]['legs'][1]['interest']['dayCountMethod'],
                             dd[i]['legs'][1]['interest'].get('rate')
                            ]], 
                             columns=col, index=[dd[i]['name']])
        pdswap = pd.concat([pdswap, pdnew], ignore_index=False)
print(j)
pdswap

/tmp/ipykernel_36055/1106541302.py:22: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  pdswap = pd.concat([pdswap, pdnew], ignore_index=False)


15


,maturityDate,quotationdate,quotationvalue,side0,ratetype,frequency,index,side1,ratetype1,frequency,rate
SW20251113_SOFR O/4.2587,2025-11-13,2024-12-19,100.016363,RECEIVE,FLOATING,ANNU,5967593,PAY,FIXED,ANNU,4.2587
SW20250211_SOFR O/4.46,2025-02-11,2024-12-19,100.060533,RECEIVE,FLOATING,ANNU,5967593,PAY,FIXED,ANNU,4.4600
SW20250407_SOFR O/4.5635,2025-04-07,2024-12-19,100.027895,RECEIVE,FLOATING,ANNU,5967593,PAY,FIXED,ANNU,4.5635
SW20250606_SOFR O/4.391,2025-06-06,2024-12-19,100.102703,RECEIVE,FLOATING,ANNU,5967593,PAY,FIXED,ANNU,4.3910
SW20250508_SOFR O/4.411,2025-05-08,2024-12-19,99.965975,RECEIVE,FLOATING,ANNU,5967593,PAY,FIXED,ANNU,4.4110
SW20251114_SOFR O/4.219,2025-11-14,2024-12-19,100.033389,RECEIVE,FLOATING,ANNU,5967593,PAY,FIXED,ANNU,4.2190
SW20250626_SOFR O/5.0435,2025-06-26,2024-12-19,99.684772,RECEIVE,FLOATING,ANNU,5967593,PAY,FIXED,ANNU,5.0435
SW20250310_SOFR O/4.6098,2025-03-10,2024-12-19,100.018657,RECEIVE,FLOATING,ANNU,5967593,PAY,FIXED,ANNU,4.6098
SW20251002_SOFR O/3.834,2025-10-02,2024-12-19,100.523361,RECEIVE,FLOATING,ANNU,5967593,PAY,FIXED,ANNU,3.8340
SW20250718_SOFR O/4.0145,2025-07-18,2024-12-19,100.366475,RECEIVE,FLOATING,ANNU,5967593,PAY,FIXED,ANNU,4.0145


In [9]:
# list of swaps no SOFR and ESTR and EU
j=0
col=['maturityDate', 'quotationdate', 'quotationvalue', 'side0', 'ratetype', 'frequency',  'rate', 'side1','ratetype1','frequency',  'index' ] 
#col=['maturityDate', 'side0', 'ratetype', 'frequency', 'daycountmethod' ,'index','ratetype1','frequency', 'daycountmethod', 'rate'] #

pdswap = pd.DataFrame(columns=col)
for i in range(len(dd)):
    if ((dd[i]['assetType']=='SWAP_ON_RATE') and ("SOFR" not in dd[i]['name']) and ("_ESTR" not in dd[i]['name'])and ("EUR" not in dd[i]['name'])):
        j=j+1
        pdnew=pd.DataFrame([[dd[i]['maturityDate'],
                             dd[i]['quotations'][0]['date'],
                             dd[i]['quotations'][0]['close'],
                             dd[i]['legs'][0]['side'], 
                             dd[i]['legs'][0]['interest']['rateType'], 
                             dd[i]['legs'][0]['interest']['paymentFrequency']['code'], 
                             #dd[i]['legs'][0]['interest']['dayCountMethod'], 
                             #dd[i]['legs'][0]['interest']['index']['_id'], 
                             dd[i]['legs'][0]['interest'].get('rate'),
                             dd[i]['legs'][1]['side'], 
                             dd[i]['legs'][1]['interest']['rateType'], 
                             dd[i]['legs'][1]['interest']['paymentFrequency']['code'], 
                             #dd[i]['legs'][1]['interest']['dayCountMethod'],
                             dd[i]['legs'][1]['interest']['index']['_id'], 
                            ]], 
                             columns=col, index=[dd[i]['name']])        
        pdswap = pd.concat([pdswap, pdnew], ignore_index=False)
print(j)
pdswap

2


/tmp/ipykernel_36055/4176899959.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  pdswap = pd.concat([pdswap, pdnew], ignore_index=False)


,maturityDate,quotationdate,quotationvalue,side0,ratetype,frequency,rate,side1,ratetype1,frequency,index
SW20260126_2.172/ESTR O,2026-01-24,2024-12-19,100.114127,RECEIVE,FIXED,ANNU,2.172,PAY,FLOATING,ANNU,6741831
SW20280325_5.75/HUF BU,2028-03-25,2024-12-19,98.812563,RECEIVE,FIXED,ANNU,5.750,PAY,FLOATING,SEME,549433


In [10]:
# list of swaps no SOFR and ESTR and EU
j=0
col=['maturityDate', 'quotationdate', 'quotationvalue', 'side0', 'ratetype', 'frequency', 'index', 'side1','ratetype1','frequency','rate' ] 
#col=['maturityDate', 'side0', 'ratetype', 'frequency', 'daycountmethod' ,'index','ratetype1','frequency', 'daycountmethod', 'rate'] #

pdswap = pd.DataFrame(columns=col)
for i in range(len(dd)):
    if ((dd[i]['assetType']=='SWAP_ON_RATE') and ("SOFR" not in dd[i]['name']) and ("_ESTR" not in dd[i]['name'])and ("EUR"  in dd[i]['name'])):
        j=j+1
        pdnew=pd.DataFrame([[dd[i]['maturityDate'],
                             dd[i]['quotations'][0]['date'],
                             dd[i]['quotations'][0]['close'],
                             dd[i]['legs'][0]['side'], 
                             dd[i]['legs'][0]['interest']['rateType'], 
                             dd[i]['legs'][0]['interest']['paymentFrequency']['code'], 
                             #dd[i]['legs'][0]['interest']['dayCountMethod'], 
                             dd[i]['legs'][0]['interest']['index']['_id'], 
                             dd[i]['legs'][1]['side'], 
                             dd[i]['legs'][1]['interest']['rateType'], 
                             #dd[i]['legs'][1]['interest']['paymentFrequency']['code'], 
                             dd[i]['legs'][1]['interest']['dayCountMethod'],
                             dd[i]['legs'][1]['interest'].get('rate'),
                            ]], 
                             columns=col, index=[dd[i]['name']])        
        pdswap = pd.concat([pdswap, pdnew], ignore_index=False)
print(j)
pdswap

2


/tmp/ipykernel_36055/3107746065.py:25: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  pdswap = pd.concat([pdswap, pdnew], ignore_index=False)


,maturityDate,quotationdate,quotationvalue,side0,ratetype,frequency,index,side1,ratetype1,frequency,rate
SW20290619_EUR EU/2.8187,2029-06-19,2024-12-19,96.071931,RECEIVE,FLOATING,SEME,251377,PAY,FIXED,5.0,2.8187
SW20290805_EUR EU/2.5602,2029-08-05,2024-12-19,99.030237,RECEIVE,FLOATING,SEME,251377,PAY,FIXED,5.0,2.5602


In [11]:
# index
j=0
col=['index' , 'assetType', 'dates', 'last', 'currency'] 
#col=['maturityDate', 'side0', 'ratetype', 'frequency', 'daycountmethod' ,'index','ratetype1','frequency', 'daycountmethod', 'rate'] #

pdswap = pd.DataFrame(columns=col)
for i in range(len(dd)):
    if ((dd[i]['_id']=='251377')or (dd[i]['_id']=='6741831')or (dd[i]['_id']=='549433')or (dd[i]['_id']=='5967593')or (dd[i]['_id']=='6741831')):
        j=j+1
        pdnew=pd.DataFrame([[dd[i]['_id'],
                             dd[i]['assetType'], 
                             dd[i]['quotations'][0]['date'], 
                             dd[i]['quotations'][0]['last'], 
                             dd[i]['ccy']
                             #dd[i]['legs'][0]['interest']['index']['_id'], 
                             #dd[i]['legs'][1]['side'], 
                             #dd[i]['legs'][1]['interest']['rateType'], 
                             #dd[i]['legs'][1]['interest']['paymentFrequency']['code'], 
                             #dd[i]['legs'][1]['interest']['dayCountMethod'],
                             #dd[i]['legs'][1]['interest'].get('rate'),
                            ]], 
                             columns=col, index=[dd[i]['name']])        
        pdswap = pd.concat([pdswap, pdnew], ignore_index=False)
        #print(dd[i])
print(j)
pdswap

4


/tmp/ipykernel_36055/2197306129.py:23: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  pdswap = pd.concat([pdswap, pdnew], ignore_index=False)


,index,assetType,dates,last,currency
EURIBOR 6M,251377,MONEY_MARKET_INDEX,2024-12-19,2.637000,EUR
HUF 6 MOIS BUBOR,549433,MONEY_MARKET_INDEX,2024-12-19,6.480000,HUF
SOFR OVERNIGHT RATE,5967593,MONEY_MARKET_INDEX,2024-12-18,4.570000,USD
ESTR OIS,6741831,MONEY_MARKET_INDEX_OIS,2024-12-20,3.739111,EUR
